## Download dataset

In [1]:
import os 

if os.path.exists("/content/dataset/img_dir/ISIC2018_Task1-2_Training_Input") == False:
    !wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip
    !unzip /content/ISIC2018_Task1-2_Training_Input.zip
    !wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Test_Input.zip
    !unzip /content/ISIC2018_Task1-2_Test_Input.zip
    !wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Test_GroundTruth.zip
    !unzip /content/ISIC2018_Task1_Test_GroundTruth.zip
    !wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip
    !unzip /content/ISIC2018_Task1_Training_GroundTruth.zip
    !wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Validation_Input.zip
    !unzip /content/ISIC2018_Task1-2_Validation_Input.zip
    !wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Validation_GroundTruth.zip
    !unzip /content/ISIC2018_Task1_Validation_GroundTruth.zip

    !rm ISIC2018_Task1-2_Test_Input.zip ISIC2018_Task1-2_Training_Input.zip ISIC2018_Task1_Test_GroundTruth.zip  ISIC2018_Task1_Training_GroundTruth.zip

    !mkdir dataset
    %cd dataset
    !mkdir img_dir
    !mkdir ann_dir
    %cd /content
    !mv /content/ISIC2018_Task1-2_Training_Input /content/dataset/img_dir
    !mv /content/ISIC2018_Task1-2_Test_Input /content/dataset/img_dir
    !mv /content/ISIC2018_Task1-2_Validation_Input /content/dataset/img_dir

    !mv /content/ISIC2018_Task1_Training_GroundTruth /content/dataset/ann_dir
    !mv /content/ISIC2018_Task1_Test_GroundTruth /content/dataset/ann_dir
    !mv /content/ISIC2018_Task1_Validation_GroundTruth /content/dataset/ann_dir


## Install MMSegmentation

In [2]:
# Install PyTorch
!conda install pytorch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0 cudatoolkit=11.3 -c pytorch
# Install mim
!pip install -U openmim
# Install mmengine
!mim install mmengine
# Install MMCV
!mim install 'mmcv >= 2.0.0rc1'

/bin/bash: conda: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html


In [3]:
!rm -rf mmsegmentation
!git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .
# %cd ../

Cloning into 'mmsegmentation'...
remote: Enumerating objects: 14947, done.
remote: Counting objects: 100% (525/525), done.
remote: Compressing objects: 100% (348/348), done.
remote: Total 14947 (delta 192), reused 350 (delta 160), pack-reused 14422
Receiving objects: 100% (14947/14947), 20.63 MiB | 30.39 MiB/s, done.
Resolving deltas: 100% (10491/10491), done.
/content/mmsegmentation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmsegmentation
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmsegmentation


In [4]:
# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.0.0


In [5]:
%cd ../
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

/content
fatal: destination path 'mmdetection' already exists and is not an empty directory.
/content/mmdetection
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmdetection
  Preparing metadata (setup.py) ... done
  Attempting uninstall: mmdet
    Found existing installation: mmdet 3.0.0
    Uninstalling mmdet-3.0.0:
      Successfully uninstalled mmdet-3.0.0
  Running setup.py develop for mmdet


In [6]:
import mmdet
print(mmdet.__version__)

3.0.0


## Customize dataset

In [7]:
!pwd
%cd ../

/content/mmdetection
/content


In [8]:
classes = ('outside', 'inside')
palette = [[0, 0, 0], [255, 255, 255]]

In [9]:
from mmseg.registry import DATASETS
from mmseg.datasets import BaseSegDataset

classes = ('outside', 'inside')
palette = [[0, 0, 0], [255, 255, 255]]

@DATASETS.register_module()
class ISICTask1Dataset(BaseSegDataset):
  METAINFO = dict(classes = classes, palette = palette)
  def __init__(self, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='_segmentation.png', **kwargs)

In [10]:
%cd mmsegmentation

/content/mmsegmentation


## Config

In [11]:
# https://github.com/open-mmlab/mmsegmentation/blob/main/configs/knet/knet-s3_r50-d8_deeplabv3_8xb2-adamw-80k_ade20k-512x512.py

from mmengine import Config
cfg = Config.fromfile('configs/knet/knet-s3_r50-d8_deeplabv3_8xb2-adamw-80k_ade20k-512x512.py')
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'ADE20KDataset'
data_root = 'data/ade/ADEChallengeData2016'
crop_size = (512, 512)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', reduce_zero_label=True),
    dict(
        type='RandomResize',
        scale=(2048, 512),
        ratio_range=(0.5, 2.0),
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(512, 512), cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='PackSegInputs')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(2048, 512), keep_ratio=True),
    dict(type='LoadAnnotations', reduce_zero_label=True),
    dict(type='PackSegInputs')
]
img_ratios = [0.5, 0.75, 1.0, 1.25, 1.5, 1.75]
tta_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(
        type='TestTimeAug',
        transforms=[[{
            'type': 'Resize',
            'scale_factor': 0.5,
            'keep_ratio':

In [12]:
# modify num classes of the model in decode head
cfg.model.decode_head.kernel_update_head[0].num_classes = 2
cfg.model.decode_head.kernel_generate_head.num_classes = 2
cfg.model.decode_head.kernel_generate_head.num_classes = 2


# Modify dataset type and batch
cfg.dataset_type = 'ISICTask1Dataset'

cfg.data_root = '/content/dataset'

cfg.train_dataloader.batch_size = 2
cfg.train_dataloader.num_workers = 2

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path='img_dir/ISIC2018_Task1-2_Training_Input', seg_map_path='ann_dir/ISIC2018_Task1_Training_GroundTruth')
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path='img_dir/ISIC2018_Task1-2_Validation_Input', seg_map_path='ann_dir/ISIC2018_Task1_Validation_GroundTruth')
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline

cfg.test_dataloader.dataset.type = cfg.dataset_type
cfg.test_dataloader.dataset.data_root = cfg.data_root
cfg.test_dataloader.dataset.data_prefix = dict(img_path='img_dir/ISIC2018_Task1-2_Test_Input', seg_map_path='ann_dir/ISIC2018_Task1_Test_GroundTruth')
cfg.test_dataloader.dataset.pipeline = cfg.test_pipeline

cfg.load_from = "https://download.openmmlab.com/mmsegmentation/v0.5/knet/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k_20220228_041642-00c8fbeb.pth"

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'
cfg.train_cfg = dict(
    type='IterBasedTrainLoop', max_iters=300, val_interval=300)


cfg.param_scheduler = [
    dict(type='LinearLR', by_epoch=False, start_factor=0.1, begin=0, end=100),
    dict(
        type='PolyLR',
        eta_min=0.0001,
        power=0.9,
        begin=0,
        end=160,
        by_epoch=False)
]

cfg['randomness'] = dict(seed=32)

In [13]:
from mmengine.runner import Runner

runner = Runner.from_cfg(cfg)

05/10 14:55:43 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 32
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm

/content/mmsegmentation/mmseg/models/backbones/resnet.py:431: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '
/content/mmsegmentation/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/content/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with

05/10 14:55:50 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
05/10 14:55:50 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

/content/mmsegmentation/mmseg/engine/hooks/visualization_hook.py:61: UserWarning: The draw is False, it means that the hook for visualization will not take effect. The results will NOT be visualized or stored.
  warnings.warn('The draw is False, it means that the '


In [14]:
runner.train()

05/10 14:55:54 - mmengine - WARNING - The prefix is not set in metric class IoUMetric.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


05/10 14:55:55 - mmengine - INFO - load model from: open-mmlab://resnet50_v1c
05/10 14:55:55 - mmengine - INFO - Loads checkpoint by openmmlab backend from path: open-mmlab://resnet50_v1c
05/10 14:55:55 - mmengine - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmsegmentation/v0.5/knet/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k_20220228_041642-00c8fbeb.pth
The model and loaded state dict do not match exactly

size mismatch for decode_head.kernel_generate_head.conv_seg.weight: copying a param with shape torch.Size([150, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 512, 1, 1]).
size mismatch for decode_head.kernel_generate_head.conv_seg.bias: copying a param with shape torch.Size([150]) from checkpoint, the shape in current model is torch.Size([2])

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


OutOfMemoryError: ignored

In [ ]:
from mmseg.apis import init_model, inference_model, show_result_pyplot

config_path = '/content/mmsegmentation/work_dirs/tutorial/pointrend_r50_4xb4-160k_ade20k-512x512.py'
checkpoint_path = "/content/mmsegmentation/work_dirs/tutorial/iter_300.pth"
img_path = '/content/dataset/img_dir/ISIC2018_Task1-2_Test_Input/ISIC_0012169.jpg'

# build the model from a config file and a checkpoint file
model = init_model(cfg, checkpoint_path, device='cuda:0')

# inference on given image
result = inference_model(model, img_path)

# display the segmentation result
vis_image = show_result_pyplot(model, img_path, result, out_file='work_dirs/result.png')

In [15]:
runner.test()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


OutOfMemoryError: ignored